In [2]:
## jupyter notebook に最初に記述tenplate

"""
2021.09.27 今回参考にしたものについて
https://qiita.com/ku_a_i/items/0ea4b93d767ce7c83145

2021.09.29 - Res-Net34について
https://pytorch.org/hub/pytorch_vision_resnet/

"""
### ------------------------------------------------
# basic-module
import matplotlib.pyplot as plt
import sys,os,re,glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
import seaborn as sns
# sori -module
# sys.path.append('/home/ysorimachi/tool')
# from getErrorValues import me,rmse,mae,r2 #(x,y)
# from convSokuhouData import conv_sfc #(df, ave=minutes,hour)

# #---------------------------------------------------
# import subprocess
# import requests
# #--------------------------
# import pickle
# import gzip
# from pathlib import Path

# deep learning modules 
# imports

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torchvision
from torchvision import transforms
from torchinfo import summary #torchinfoはニューラルネットの中身を見れるのでおすすめ

In [3]:
#参考の資料について
# https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

def load_df(path, isDAY=True):
    df = pd.read_csv(path)
    df["time"] = pd.to_datetime(df["time"])
    df["dd"] = df["time"].apply(lambda x: x.strftime("%Y%m%d"))
    df["hh"] = df["time"].apply(lambda x: x.strftime("%H"))
    
    if isDAY:
        df = df.groupby("dd").agg({"DEMAND(10^4kW)" : "mean"})
    return df

def plot_df(df,_col):
    f,ax = plt.subplots(figsize=(18,8))
    for c in _col:
        ax.plot(df[c],label=c)
    
    ax.legend(loc="upper right")
    plt.show()
    return

def cut_time(df,st,ed=None):
    if ed:
        return df[(df["time"]>=st)&(df["time"]<=ed)]
    else:
        return df[df["time"]>=st]

def move_ave(df,col,lag=7):
    new_col  = f"MA{lag}_{col}"
    df[new_col]= df[col].rolling(lag).mean()
    return df

In [4]:
"""
import torch ResNet pretrained model 
"""
import torchvision
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# model = torchvision.models.resnet18(pretrained= True)
# model = torchvision.models.resnet34(pretrained= True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
def load_resnet(N=18,pretrained=True):
    if N==18:
        return torchvision.models.resnet18(pretrained= pretrained)
    if N==34:
        return torchvision.models.resnet34(pretrained= pretrained)


In [5]:
def pic_imshow(img):
    if type(img)== torch.Tensor:
        img = img.to('cpu').detach().numpy()
        if img.ndim ==3:
            nx,ny = img.shape[1:]
            img = img[0,:,:].reshape(nx,ny)
    f,ax = plt.subplots(figsize=(10,10))
    h,w = img.shape[:2]
    ax.set_title(f"img({h}pix*{w}pix)")
    ax.imshow(img)
    plt.show()
    return


import urllib
import cv2
import io #Pillowを使って、URLから画像を読みこむ
# https://qiita.com/zabeth129/items/b355ebfc82d38bc49778
from  PIL import Image

def load_sample(ndarray=False):
    path = "https://github.com/pytorch/hub/raw/master/images/dog.jpg"
#     img = cv2.imread(path)
    res = urllib.request.urlopen(path)
    if res.code ==200:
        f_obj = io.BytesIO(res.read())
        img = Image.open(f_obj)
        if ndarray:
            img = np.array(img)
        return img
    else:
        print("Not Data !",res.code)
        return None

#     url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
#     try:
#         urllib.URLopener().retrieve(url, filename)
#     except: 
#         urllib.request.urlretrieve(url, filename)
#     return filename

# img = load_sample()
# img2 = preprocess(img)
# pic_imshow(img2)
# img2.size()
# type(img2)

In [21]:
#-----------------------------------------
import netCDF4
from netCDF4 import Dataset
class OpenNetCDF:
    DIR_DATA="/work/ysorimachi/era5/dat2"
    
    def __init__(self):
        self.cele = None
        self.mm = None
        self.nc = None
        
    def path(self):
    #     high_cloud_cover  low_cloud_cover  mean_sea_level_pressure  medium_cloud_cover
        if self.cele == "hc":
            name = "high_cloud_cover"
        elif self.cele == "mc":
            name = "medium_cloud_cover"
        elif self.cele == "lc":
            name = "low_cloud_cover"
        elif self.cele == "sp":
            name = "mean_sea_level_pressure"
            
        path = f"{self.DIR_DATA}/{name}/download_{self.mm}.nc"
        return path
    
    def load(self,cele,mm):
        self.cele = cele
        self.mm =mm
        
        path = self.path()
        if path is not None:
            nc = Dataset(path, 'r')
            self.nc = nc
            
            data = self._data
            if type(data) == netCDF4._netCDF4.Variable:
                data = np.array(data)
            time = self._time
            _lat = self._lat
            _lon = self._lon
            return data,time,_lat,_lon
            
        else:
            print("Not Found !")
    
    def load_multi(self,cele,_mm):
        _data =[]
        _time = []
        for mm in tqdm(_mm):
            data,time,_lat,_lon = self.load(cele,mm)
            _time += time
            _data.append(data)
        
        data = np.concatenate(_data)
        return data,_time,_lat,_lon
    
#     @property
    def check(self,cele,mm):
        self.cele = cele
        self.mm =mm
        path = self.path()
        if path is not None:
            nc = Dataset(path, 'r')
        return nc.variables.keys()
    
    @property
    def _data(self):
        if self.cele == "hc":
            k = "hcc"
        elif self.cele == "mc":
            k = "mcc"
        elif self.cele == "lc":
            k = "lcc"
        elif self.cele == "sp":
            k = "msl"
        return self.nc.variables[k]
    
    @property
    def _time(self):
        _t = list(self.nc.variables["time"])
        _t = [ self.conv_time(t) for t in _t ]
        return _t
    
    def conv_time(self,t):
        init = datetime(1900,1,1,0,0)
        return init + timedelta(hours=int(t.data))
    
    @property
    def _lon(self):
        _lon = [ float(t.data) for t in self.nc.variables["longitude"] ]
        return _lon
    @property
    def _lat(self):
        _lat = [ float(t.data) for t in self.nc.variables["latitude"] ]
        return _lat

def loop_mm(yy=5):
    _mm=[]
    for yy in range(2014,2014+yy):
        for mm in range(1,12+1):
            cmm= str(mm).zfill(2)
            _mm.append(f"{yy}{cmm}")
    print("N_month ->", len(_mm))
    return _mm

# def_path("hc","202012")
nc = OpenNetCDF()
_mm=loop_mm()[:3]

# data,_t,_lon,_lat = nc.load_multi("sp",_mm)
# data.shape,len(_t)

 67%|██████▋   | 2/3 [00:00<00:00, 11.22it/s]

N_month -> 60


100%|██████████| 3/3 [00:00<00:00, 10.70it/s]


((720, 121, 121), 720)

In [39]:
def random_idx(st,ed,N):
    return sorted(list(np.random.randint(st,ed,N)))

def get_time(_idx,_time):
    return [ _t[i] for i in _idx]
_mm=loop_mm()[:5]


 40%|████      | 2/5 [00:00<00:00, 13.81it/s]

N_month -> 60


100%|██████████| 5/5 [00:00<00:00, 12.93it/s]


In [155]:
def check_mm(cele):
    DIR_DATA="/work/ysorimachi/era5/dat2"
    if cele == "hc":
        name = "high_cloud_cover"
    elif cele == "mc":
        name = "medium_cloud_cover"
    elif cele == "lc":
        name = "low_cloud_cover"
    elif cele == "sp":
        name = "mean_sea_level_pressure"
    
    _path = sorted(os.listdir(f"{DIR_DATA}/{name}"))
    
    _yy = [ f.split("_")[1][:6] for f in _path] 
    return list(_yy)


In [60]:
# pic_imshow(spl[5,:,:])
def pic_imshow35(img,list_time,_idx):
    _t = get_time(_idx,list_time)
    _t = [ t.strftime("%Y%m%d %H:%M") for t in _t]
    
    f,ax = plt.subplots(5,7,figsize=(7*4,5*4))
    ax = ax.flatten()
    for i,idx in enumerate(_idx):
        img2 = img[idx,:,:]
#         print(img2.shape)
        ax[i].imshow(img2,vmin=0, vmax=1,cmap="seismic")
        h,w = img2.shape
        ct = _t[i]
        ax[i].set_title(f"img({h}*{w}) {ct}")
    plt.show()
    return 


# data,_t,_lon,_lat = nc.load_multi("sp",_mm)
_idx = random_idx(0,data.shape[0],35)
# pic_imshow35(data,_t,_idx)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
class Scaler:
    def __init__(self,name):
        self.sc = {}
        self.name = name
    
    def fit_transform(self,data):
        for i in range(data.shape[0]):
                if self.name == "minmax":
                    self.sc[i] = MinMaxScaler()
                if self.name == "std":
                    self.sc[i] = StandardScaler()
                
                data[i,:,:] = self.sc[i].fit_transform(data[i,:,:])
        
        return data

(0.0, 1.0)

In [80]:
"""
model の定義でCNNでECMWFの画像について実装していく予定
"""
class CNN_ENCODER(nn.Module):
    def __init__(self, out_size=16):
        super(CNN_ENCODER,self).__init__()
        # 入力のinput画像について　56*56で入力
        #----------------
        #1chan * 56x56　->　8chan * 28x28
        self.l1 = nn.Sequential(
            nn.Conv2d(1,8,kernel_size=5, padding=2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2))
        #----------------
        #8chan * 28x28　->　16chan * 14x14
        self.l2 = nn.Sequential(
            nn.Conv2d(8,16,kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        #----------------
        #16chan * 14x14 -> 32 * 7x7
        self.l3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        #----------------
        #Linear 2 layer
        self.l4 = nn.Sequential(
            nn.Linear(32*7*7,512),
            nn.ReLU(),
            nn.Linear(512,128))
        
    def forward(self,x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        
        x = x.view(x.size(0),-1)
        x = self.l4(x)
        return x

    
def ResNET(N=18,pretrained=True):
    if N==18:
        return torchvision.models.resnet18(pretrained= pretrained)
    if N==34:
        return torchvision.models.resnet34(pretrained= pretrained)


In [62]:
#summary
# model = ResNET(34)
model = CNN_ENCODER()
res18 = ResNET(18)
res34 = ResNET(34)
# model

In [65]:
summary(
    model,
    input_size=(16,1,56, 56),
    col_names=["output_size", "num_params"]
)

Layer (type:depth-idx)                   Output Shape              Param #
CNN_ENCODER                              --                        --
├─Sequential: 1-1                        [16, 8, 28, 28]           --
│    └─Conv2d: 2-1                       [16, 8, 56, 56]           208
│    └─BatchNorm2d: 2-2                  [16, 8, 56, 56]           16
│    └─ReLU: 2-3                         [16, 8, 56, 56]           --
│    └─MaxPool2d: 2-4                    [16, 8, 28, 28]           --
├─Sequential: 1-2                        [16, 16, 14, 14]          --
│    └─Conv2d: 2-5                       [16, 16, 28, 28]          3,216
│    └─BatchNorm2d: 2-6                  [16, 16, 28, 28]          32
│    └─ReLU: 2-7                         [16, 16, 28, 28]          --
│    └─MaxPool2d: 2-8                    [16, 16, 14, 14]          --
├─Sequential: 1-3                        [16, 32, 7, 7]            --
│    └─Conv2d: 2-9                       [16, 32, 14, 14]          12,832
│    └─

In [162]:
def preprocess(img):
    pre_prs = transforms.Compose([
        transforms.Resize(70),
        transforms.CenterCrop(56),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         transforms.Grayscale(num_output_channels=1),
    ])
    img = pre_prs(img)
    # Min -Max Scaler
#     https://discuss.pytorch.org/t/how-to-efficiently-normalize-a-batch-of-tensor-to-0-1/65122
    
    return img

def mk_DataLoader(cele,_mm):
    nc = OpenNetCDF()
    sc = Scaler("minmax")
        
    data,_time,_,_ = nc.load_multi(cele,_mm)
    data = sc.fit_transform(data)
    
    if type(data) != np.ndarray:
        data = np.array(data)
    
    _img=[]
    for i in range(data.shape[0]):
        img = data[i,:,:]
        
        if type(img) != Image.Image:
            img = Image.fromarray(img)
        img = preprocess(img)
        _img.append(img)
    
    _img = tuple(_img)
    _img = torch.stack(_img,0)
    return _img,_time

# _img,_time = mk_DataLoader("sp",_mm)

In [72]:
_img = tuple(_img)
_img = torch.stack(_img,0)
_img.size()

torch.Size([720, 1, 56, 56])

In [81]:

_img,_time = mk_DataLoader("sp",N=3)
model = CNN_ENCODER()
res = model(_img)
res2 = res.detach().numpy()
res2.shape

(720, 128)

In [91]:
sys.path.append("/home/ysorimachi/work/sori_py2/deepl/py")

In [92]:
sys.path

['/home/ysorimachi/work/sori_py2/deepl/jupyter',
 '/home/ysorimachi/.conda/envs/sori_torch/lib/python38.zip',
 '/home/ysorimachi/.conda/envs/sori_torch/lib/python3.8',
 '/home/ysorimachi/.conda/envs/sori_torch/lib/python3.8/lib-dynload',
 '',
 '/home/ysorimachi/.local/lib/python3.8/site-packages',
 '/home/ysorimachi/.conda/envs/sori_torch/lib/python3.8/site-packages',
 '/home/ysorimachi/.conda/envs/sori_torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/ysorimachi/.ipython',
 'py',
 '/home/ysorimachi/work/sori_py2/deep/py',
 '/home/ysorimachi/work/sori_py2/deepl/py']

In [94]:
from som2 import SOM
# os.listdir("/home/ysorimachi/work/sori_py2/deep/py")

In [148]:
from som2 import SOM
from sklearn.cluster import KMeans,AgglomerativeClustering
# from (self, m=3, n=3, dim=3, lr=1, sigma=1,distance="ED",max_iter=3000, epoch=1)
class ClusterModel:
    def __init__(self,method,n_clusters=16):
        self.method = method
        map_size = int(np.sqrt(n_clusters))
        
        if method=="kmeans":
            m = KMeans(n_clusters=n_clusters)
        if method=="ward":
            m = AgglomerativeClustering(n_clusters=n_clusters)
        if method =="SOM-ED":
            m = SOM(m=map_size, n=map_size,dim=128,distance="ED")
        if method == "SOM-SSIM":
            m = SOM(m=map_size, n=map_size,dim=128,distance="SSIM")
        self.model = m
        self.isTrained = 0
        
    def fit(self,X):
        self.model.fit(X)
        self.isTrained = 1
        self.X = X
        
    @property
    def labels(self):
        if self.method=="kmeans" or self.method=="ward":
            lbl = pd.Series(self.model.labels_,name = "labels")
        else:
#             print("SOM")
            lbl = self.model.predict(self.X)
            lbl = pd.Series(lbl,name="labels")
        return lbl 
        
    def predict(self,X):
        if self.isTrained == 1:
            pred = self.model.predict(X)
        else:
            self.fit(X)
            pred = self.model.predict(X)
            
        pred = pd.Series(pred,name = "pred")
        return pred


In [149]:
m = ClusterModel("ward")
m.fit(res2)
m.labels

0       2
1       2
2       2
3       2
4       2
       ..
715    13
716    13
717    13
718    13
719    13
Name: labels, Length: 720, dtype: int64

In [110]:
_img,_time = mk_DataLoader("sp",N=3)
model = CNN_ENCODER()
res = model(_img)
res2 = res.detach().numpy()
res2.shape

In [163]:
_mm = check_mm("sp")[:12*18]
_img,_time = mk_DataLoader("sp",_mm)
_img.size()

100%|██████████| 240/240 [00:27<00:00,  8.67it/s]


torch.Size([58440, 1, 56, 56])

In [164]:
model = CNN_ENCODER()
res = model(_img)
res2 = res.detach().numpy()

In [184]:
import pickle
DIR="/home/ysorimachi/work/ecmwf/out/cluster210930/label/cnn"
def loop_train(n_month=6):
    for cate in [ "sp","hc","mc","lc"]:
        _mm = check_mm(cate)[:n_month]
        _img,_time = mk_DataLoader(cate,_mm)
        #model create---
        model = CNN_ENCODER()
        res = model(_img) # forward calc
        res2 = res.detach().numpy()
#         print(cate, len(_mm))
        #------------
        _model,_lbl=[],[]
        df = pd.DataFrame()
        df["time"] =_time

        for name in tqdm(["kmeans","ward","SOM-ED","SOM-SSIM"]):
            m = ClusterModel(name)
            m.fit(res2)
            lbl = m.labels
            lbl.name = name
            
            #model save
            model_path = f"{DIR}/cluster_{cate}_{name}.pkl"
            with open(model_path, 'wb') as pkl:
                pickle.dump(m, pkl)
            _lbl.append(lbl)

        df = pd.concat([df] + _lbl,axis=1)
        df.to_csv(f"{DIR}/label_{cate}.csv",index=False)
        
loop_train(n_month=12*19)

100%|██████████| 4/4 [10:41<00:00, 160.44s/it]
